In [1]:
pip install pytorch-tabnet

  Using cached threadpoolctl-3.5.0-py3-none-any.whl.metadata (13 kB)
   ---------------------------------------- 0.0/11.1 MB ? eta -:--:--
   -- ------------------------------------- 0.8/11.1 MB 4.8 MB/s eta 0:00:03
   ------ --------------------------------- 1.8/11.1 MB 4.8 MB/s eta 0:00:02
   --------- ------------------------------ 2.6/11.1 MB 4.4 MB/s eta 0:00:02
   -------------- ------------------------- 3.9/11.1 MB 4.9 MB/s eta 0:00:02
   ------------------ --------------------- 5.2/11.1 MB 5.1 MB/s eta 0:00:02
   --------------------- ------------------ 6.0/11.1 MB 4.9 MB/s eta 0:00:02
   ------------------------- -------------- 7.1/11.1 MB 4.8 MB/s eta 0:00:01
   --------------------------- ------------ 7.6/11.1 MB 4.6 MB/s eta 0:00:01
   ------------------------------- -------- 8.7/11.1 MB 4.6 MB/s eta 0:00:01
   --------------------------------- ------ 9.4/11.1 MB 4.5 MB/s eta 0:00:01
   ------------------------------------ --- 10.2/11.1 MB 4.5 MB/s eta 0:00:01
   ----------


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: C:\Users\maxim\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
import numpy as np
import pandas as pd

In [4]:
pd.read_csv('migraine_data.csv')

,Age,Duration,Frequency,Location,Character,Intensity,Nausea,Vomit,Phonophobia,Photophobia,...,Vertigo,Tinnitus,Hypoacusis,Diplopia,Defect,Ataxia,Conscience,Paresthesia,DPF,Type
0,30,1,5,1,1,2,1,0,1,1,...,0,0,0,0,0,0,0,0,0,Typical aura with migraine
1,50,3,5,1,1,3,1,1,1,1,...,1,0,0,0,0,0,0,0,0,Typical aura with migraine
2,53,2,1,1,1,2,1,1,1,1,...,0,0,0,0,0,0,0,0,0,Typical aura with migraine
3,45,3,5,1,1,3,1,0,1,1,...,1,0,0,0,0,0,0,0,0,Typical aura with migraine
4,53,1,1,1,1,2,1,0,1,1,...,0,0,0,0,0,0,0,0,1,Typical aura with migraine
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,39,1,1,1,1,3,1,1,1,1,...,1,0,0,1,0,0,0,0,1,Basilar-type aura
396,20,3,2,1,1,3,1,0,1,1,...,0,1,1,0,0,0,0,0,1,Basilar-type aura
397,28,3,2,1,1,3,1,0,1,1,...,1,0,0,0,1,0,0,0,1,Basilar-type aura
398,20,3,1,1,1,3,1,0,1,1,...,1,0,0,0,0,0,0,1,1,Basilar-type aura


In [15]:
from pytorch_tabnet.tab_model import TabNetClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Step 2: Load and Inspect the Data
# Load your migraine dataset
df = pd.read_csv('migraine_data.csv')

# Display the column names to identify the target column
print("Column names in the dataset:")
print(df.columns)

print("First few rows of the dataset:")
print(df.head())

# Step 3: Set the Target Column
target_column = 'Type'  # The target column is 'Type'

# Separate features and target
X = df.drop(target_column, axis=1).values
y = df[target_column].values

# Step 4: Preprocess the Data
# Encode labels if they are categorical
le = LabelEncoder()
y = le.fit_transform(y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize the data (TabNet works best with normalized data)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Step 5: Build and Train the TabNet Model
# Initialize TabNetClassifier
tabnet_model = TabNetClassifier()

# Train the model
tabnet_model.fit(
    X_train, y_train,
    max_epochs=75,
    patience=10,
    batch_size=256,
    virtual_batch_size=128,
    num_workers=1,
    drop_last=False
)

# Step 6: Evaluate the Model
# Predict on test data
y_pred = tabnet_model.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f'Test Accuracy: {accuracy:.4f}')

# Calculate precision for each class
precision = precision_score(y_test, y_pred, average=None, labels=np.unique(y_pred))
print(f'Precision for each class: {precision}')

# Calculate the average precision
average_precision = precision_score(y_test, y_pred, average='weighted')
print(f'Average Precision: {average_precision:.4f}')

# (Optional) Feature Importance
importances = tabnet_model.feature_importances_
print("Feature Importances:", importances)

import matplotlib.pyplot as plt

# Step 7: Feature Importance
features = df.drop(target_column, axis=1).columns

# Sort the importances and corresponding features
sorted_indices = np.argsort(importances)
sorted_importances = importances[sorted_indices]
sorted_features = features[sorted_indices]

# Plot the Feature Importances on a curve
plt.figure(figsize=(10, 6))
plt.plot(sorted_features, sorted_importances, marker='o')
plt.xticks(rotation=45, ha='right')
plt.xlabel('Features')
plt.ylabel('Importance')
plt.title('Feature Importances in Migraine Classification')
plt.grid(True)
plt.tight_layout()
plt.show()


OSError: [WinError 126] The specified module could not be found. Error loading "C:\Users\maxim\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torch\lib\shm.dll" or one of its dependencies.

In [16]:
import numpy as np
import pandas as pd
from pytorch_tabnet.tab_model import TabNetClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Step 2: Load and Inspect the Data
# Load your migraine dataset
df = pd.read_csv('migraine_data.csv')

# Display the column names to identify the target column
print("Column names in the dataset:")
print(df.columns)

print("First few rows of the dataset:")
print(df.head())

# Step 3: Set the Target Column
target_column = 'Type'  # The target column is 'Type'

# Separate features and target
X = df.drop(target_column, axis=1).values
y = df[target_column].values

# Step 4: Preprocess the Data
# Encode labels if they are categorical
le = LabelEncoder()
y = le.fit_transform(y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize the data (TabNet works best with normalized data)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Step 5: Build and Train the TabNet Model
# Initialize TabNetClassifier
tabnet_model = TabNetClassifier()

# Train the model
tabnet_model.fit(
    X_train, y_train,
    max_epochs=100,
    patience=10,
    batch_size=256,
    virtual_batch_size=128,
    num_workers=1,
    drop_last=False
)

# Step 6: Evaluate the Model
# Predict on test data
y_pred = tabnet_model.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f'Test Accuracy: {accuracy:.4f}')

# Step 7: Feature Importance
importances = tabnet_model.feature_importances_
features = df.drop(target_column, axis=1).columns

# Sort the importances and corresponding features
sorted_indices = np.argsort(importances)
sorted_importances = importances[sorted_indices]
sorted_features = features[sorted_indices]

# Step 8: Implement the Migraine Type Checker with User Input
def check_migraine_type(user_input):
    # Preprocess the input data
    input_data_scaled = scaler.transform(np.array([user_input]))

    # Predict using the trained model
    prediction = tabnet_model.predict(input_data_scaled)

    # Decode the prediction to the original label
    predicted_type = le.inverse_transform(prediction)

    return predicted_type[0]

# Step 9: Ask for User Input via CLI
print("\nPlease enter the following details to predict the type of migraine:")

user_input = []
for feature in features:
    value = float(input(f"Enter value for {feature}: "))
    user_input.append(value)

predicted_migraine_type = check_migraine_type(user_input)

if predicted_migraine_type == "Other":
    print("\nBased on your input you might not have Migraine, please confirm with a doctor if symptoms prolong.")
else:
    print(f"\nPredicted Migraine Type: {predicted_migraine_type}")


OSError: [WinError 126] The specified module could not be found. Error loading "C:\Users\maxim\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torch\lib\shm.dll" or one of its dependencies.

In [9]:
pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: C:\Users\maxim\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [12]:
pip install torch

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: C:\Users\maxim\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [14]:
pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu


Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://download.pytorch.org/whl/cpu



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: C:\Users\maxim\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [13]:
import numpy as np
import pandas as pd
from pytorch_tabnet.tab_model import TabNetClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Load and Inspect the Data
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/migraine_data.csv')

# Set the Target Column
target_column = 'Type'  # The target column is 'Type'

# Separate features and target
X = df.drop(target_column, axis=1).values
y = df[target_column].values

# Preprocess the Data
le = LabelEncoder()
y = le.fit_transform(y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Build and Train the TabNet Model
tabnet_model = TabNetClassifier()

tabnet_model.fit(
    X_train, y_train,
    max_epochs=100,
    patience=10,
    batch_size=256,
    virtual_batch_size=128,
    num_workers=1,
    drop_last=False
)

# Evaluate the Model
y_pred = tabnet_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Test Accuracy: {accuracy:.4f}')

# Save the trained model, label encoder, and scaler for future use
import joblib
joblib.dump(tabnet_model, 'tabnet_model.pkl')
joblib.dump(le, 'label_encoder.pkl')
joblib.dump(scaler, 'scaler.pkl')


OSError: [WinError 126] The specified module could not be found. Error loading "C:\Users\maxim\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torch\lib\shm.dll" or one of its dependencies.

In [ ]:
import numpy as np
import joblib

# Load the trained model, label encoder, and scaler
tabnet_model = joblib.load('tabnet_model.pkl')
le = joblib.load('label_encoder.pkl')
scaler = joblib.load('scaler.pkl')

# List of features (you can hardcode this or load it dynamically from the dataset)
features = ['Age', 'Duration', 'Frequency', 'Location', 'Character', 'Intensity',
            'Nausea', 'Vomit', 'Phonophobia', 'Photophobia', 'Visual', 'Sensory',
            'Dysphasia', 'Dysarthria', 'Vertigo', 'Tinnitus', 'Hypoacusis',
            'Diplopia', 'Defect', 'Ataxia', 'Conscience', 'Paresthesia', 'DPF']

def check_migraine_type(user_input):
    # Preprocess the input data
    input_data_scaled = scaler.transform(np.array([user_input]))

    # Predict using the trained model
    prediction = tabnet_model.predict(input_data_scaled)

    # Decode the prediction to the original label
    predicted_type = le.inverse_transform(prediction)

    return predicted_type[0]

# Ask for User Input via CLI
print("\nPlease enter the following details to predict the type of migraine:")

user_input = []
for feature in features:
    value = float(input(f"Enter value for {feature}: "))
    user_input.append(value)

predicted_migraine_type = check_migraine_type(user_input)

if predicted_migraine_type == "Other":
    print("\nBased on your input you might not have Migraine, please confirm with a doctor if symptoms prolong.")
else:
    print(f"\nPredicted Migraine Type: {predicted_migraine_type}")


/usr/local/lib/python3.10/dist-packages/torch/storage.py:414: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(io.BytesIO(b))



Please enter the following details to predict the type of migraine:
Enter value for Age: 30
Enter value for Duration: 2
Enter value for Frequency: 3
Enter value for Location: 1
Enter value for Character: 3
Enter value for Intensity: 2
Enter value for Nausea: 0
Enter value for Vomit: 1
Enter value for Phonophobia: 0
Enter value for Photophobia: 0
Enter value for Visual: 0
Enter value for Sensory: 0
Enter value for Dysphasia: 0
Enter value for Dysarthria: 0
Enter value for Vertigo: 0
Enter value for Tinnitus: 0
Enter value for Hypoacusis: 1
Enter value for Diplopia: 0
Enter value for Defect: 1
Enter value for Ataxia: 0
Enter value for Conscience: 0
Enter value for Paresthesia: 0
Enter value for DPF: 0

Based on your input you might not have Migraine, please confirm with a doctor if symptoms prolong.
